# TSA Chapter 5: Full VaR Backtesting Framework

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_backtest_full/TSA_ch5_backtest_full.ipynb)

Complete backtesting framework with Kupiec and Christoffersen tests.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
import yfinance as yf
from arch import arch_model

In [ ]:
# Download S&P 500 data
sp = yf.download('^GSPC', start='2015-01-01', end='2025-12-31', progress=False)
if isinstance(sp.columns, pd.MultiIndex):
    sp.columns = sp.columns.get_level_values(0)
returns = (sp['Close'].pct_change() * 100).dropna()
n_total = len(returns); n_test = 500; n_train = n_total - n_test

In [ ]:
# Rolling GARCH VaR forecast
var_forecasts = []
for i in range(n_test):
    window = returns.iloc[:n_train + i]
    mod = arch_model(window, vol='Garch', p=1, q=1, mean='Constant', dist='t')
    r = mod.fit(disp='off', update_freq=0)
    fcast = r.forecast(horizon=1)
    sigma1 = np.sqrt(fcast.variance.values[-1, 0])
    nu = r.params.get('nu', 5)
    var_forecasts.append(-stats.t.ppf(0.01, nu) * sigma1)
    if (i+1) % 100 == 0: print(f"  {i+1}/{n_test}")

test_returns = returns.iloc[n_train:].values
var_arr = np.array(var_forecasts)
violations = test_returns < -var_arr
print(f"\nViolations: {violations.sum()}/{n_test} = {violations.mean():.4f} (expected: 0.01)")

In [ ]:
# Kupiec unconditional coverage test
n_v = violations.sum(); n_obs = n_test; p = 0.01
p_hat = n_v / n_obs
if n_v > 0 and n_v < n_obs:
    lr = -2 * (n_v * np.log(p) + (n_obs - n_v) * np.log(1 - p) -
               n_v * np.log(p_hat) - (n_obs - n_v) * np.log(1 - p_hat))
else:
    lr = 0
p_value = 1 - stats.chi2.cdf(lr, 1)
print(f"Kupiec Test: LR = {lr:.4f}, p-value = {p_value:.4f}")
print(f"Result: {'PASS' if p_value > 0.05 else 'FAIL'} at 5% significance")